In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/buliding-density/validation_data_with_density.csv
/kaggle/input/buliding-density/training_data_with_density.csv
/kaggle/input/water-feature-450m-buffer/train_df_with_water.csv
/kaggle/input/water-feature-450m-buffer/valid_df_with_water.csv
/kaggle/input/green-space-without-recreational-ground/train_df_with_green_space (4).csv
/kaggle/input/green-space-without-recreational-ground/valid_df_with_green_space (4).csv
/kaggle/input/building-density2-taller-than-18m-500m/valid_df_with_building_density2 (3).csv
/kaggle/input/building-density2-taller-than-18m-500m/train_df_with_building_density2 (3).csv
/kaggle/input/road-density-300m-without-primary/train_df_with_road_density (4).csv
/kaggle/input/road-density-300m-without-primary/valid_df_with_road_density (4).csv
/kaggle/input/new-training-with-density/uncorrupted_training_data_with_density.csv
/kaggle/input/satellite-data/validation_data_with_satellite.csv
/kaggle/input/satellite-data/training_data_with_satellite.csv
/kaggle/i

In [2]:
# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

import optuna

# Combine Datasets

In [3]:
train_density = pd.read_csv("/kaggle/input/new-training-with-density/uncorrupted_training_data_with_density.csv")
train_density.head()

,Longitude,Latitude,datetime,UHI Index,density
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8


In [4]:
train_satellite = pd.read_csv("/kaggle/input/uncorrupted-training-satellite/uncorrupted_training_data_with_satellite.csv")
train_satellite.head()

,Longitude,Latitude,datetime,UHI Index,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,825.0,990.0,1144.0,1172.0,1435.0,1688.0,1688.0,1830.0,1688.0,1788.0,1540.0,0.219187,-0.011609,-0.230666,38.393941
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,825.0,990.0,1144.0,1172.0,1435.0,1688.0,1688.0,1830.0,1688.0,1788.0,1540.0,0.219187,-0.011609,-0.230666,38.393941
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,825.0,622.0,785.0,744.0,1114.0,2006.0,2138.0,2318.0,2353.0,1566.0,1170.0,0.514043,-0.193615,-0.494038,37.785534
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,825.0,619.0,750.0,738.0,1114.0,2006.0,2138.0,2302.0,2353.0,1566.0,1170.0,0.514474,-0.190279,-0.508519,37.785534
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,825.0,556.0,758.0,660.0,1056.0,1891.0,2141.0,2280.0,2259.0,1658.0,1240.0,0.551020,-0.157948,-0.500987,37.358281


In [5]:
train_satellite = train_satellite.drop(["Longitude","Latitude","datetime","UHI Index"], axis=1)
train_concat = pd.concat([train_density, train_satellite], axis = 1)
train_concat.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,825.0,990.0,1144.0,1172.0,1435.0,1688.0,1688.0,1830.0,1688.0,1788.0,1540.0,0.219187,-0.011609,-0.230666,38.393941
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,825.0,990.0,1144.0,1172.0,1435.0,1688.0,1688.0,1830.0,1688.0,1788.0,1540.0,0.219187,-0.011609,-0.230666,38.393941
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,825.0,622.0,785.0,744.0,1114.0,2006.0,2138.0,2318.0,2353.0,1566.0,1170.0,0.514043,-0.193615,-0.494038,37.785534
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,825.0,619.0,750.0,738.0,1114.0,2006.0,2138.0,2302.0,2353.0,1566.0,1170.0,0.514474,-0.190279,-0.508519,37.785534
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,825.0,556.0,758.0,660.0,1056.0,1891.0,2141.0,2280.0,2259.0,1658.0,1240.0,0.551020,-0.157948,-0.500987,37.358281


In [6]:
validation_density = pd.read_csv("/kaggle/input/buliding-density/validation_data_with_density.csv")
validation_density.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,density
0,-73.971665,40.788763,NaN,12
1,-73.971928,40.788875,NaN,12
2,-73.967080,40.789080,NaN,4
3,-73.972550,40.789082,NaN,12
4,-73.969697,40.787953,NaN,9


In [7]:
validation_satellite = pd.read_csv("/kaggle/input/satellite-data/validation_data_with_satellite.csv")
validation_satellite.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST
0,-73.971665,40.788763,NaN,794.0,511.0,568.0,527.0,1112.0,2323.0,2649.0,2360.0,2784.0,1741.0,1218.0,0.634915,-0.150939,-0.612022,36.202990
1,-73.971928,40.788875,NaN,1221.0,494.0,661.0,497.0,1315.0,2652.0,2964.0,3662.0,3102.0,2124.0,1541.0,0.761000,-0.265814,-0.694194,36.202990
2,-73.967080,40.789080,NaN,1049.0,841.0,968.0,1056.0,1028.0,1418.0,1610.0,1402.0,1636.0,1634.0,1198.0,0.140765,0.076416,-0.183122,36.069687
3,-73.972550,40.789082,NaN,1189.0,903.0,984.0,1108.0,1971.0,1978.0,1970.0,1478.0,2049.0,2303.0,2219.0,0.143078,0.218196,-0.200650,36.886594
4,-73.969697,40.787953,NaN,1018.0,716.0,1046.0,917.0,1286.0,2382.0,2778.0,3038.0,2912.0,2102.0,1705.0,0.536283,-0.182101,-0.487757,34.500816


In [8]:
validation_satellite = validation_satellite.drop(["Longitude","Latitude","UHI Index"], axis=1)
validation_concat = pd.concat([validation_density, validation_satellite], axis=1)
validation_concat.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST
0,-73.971665,40.788763,NaN,12,794.0,511.0,568.0,527.0,1112.0,2323.0,2649.0,2360.0,2784.0,1741.0,1218.0,0.634915,-0.150939,-0.612022,36.202990
1,-73.971928,40.788875,NaN,12,1221.0,494.0,661.0,497.0,1315.0,2652.0,2964.0,3662.0,3102.0,2124.0,1541.0,0.761000,-0.265814,-0.694194,36.202990
2,-73.967080,40.789080,NaN,4,1049.0,841.0,968.0,1056.0,1028.0,1418.0,1610.0,1402.0,1636.0,1634.0,1198.0,0.140765,0.076416,-0.183122,36.069687
3,-73.972550,40.789082,NaN,12,1189.0,903.0,984.0,1108.0,1971.0,1978.0,1970.0,1478.0,2049.0,2303.0,2219.0,0.143078,0.218196,-0.200650,36.886594
4,-73.969697,40.787953,NaN,9,1018.0,716.0,1046.0,917.0,1286.0,2382.0,2778.0,3038.0,2912.0,2102.0,1705.0,0.536283,-0.182101,-0.487757,34.500816


# Add features

## 1. Enhanced Vegetation Index (EVI)
The EVI is used to measure vegetation greenness and is an improvement over NDVI, as it is less sensitive to atmospheric conditions and soil background.

$$
\text{EVI} = 2.5 \cdot \frac{(B8 - B4)}{(B8 + 6 \cdot B4 - 7.5 \cdot B2 + 1)}
$$

Where:
- \(B8\) = Near-Infrared (NIR) band (Band 8)
- \(B4\) = Red band (Band 4)
- \(B2\) = Blue band (Band 2)
- The constants are: \( G = 2.5, C_1 = 6, C_2 = 7.5, L = 1 \)

## 2. Soil Adjusted Vegetation Index (SAVI)
The SAVI is an index used to measure vegetation while adjusting for the influence of soil brightness in areas with sparse vegetation.

$$
\text{SAVI} = \frac{(B8 - B4)}{(B8 + B4 + 0.5)} \cdot (1 + 0.5)
$$

Where:
- \(B8\) = Near-Infrared (NIR) band (Band 8)
- \(B4\) = Red band (Band 4)
- The constant \(L = 0.5\) is used for intermediate vegetation cover.

## 3. Modified Normalized Difference Water Index (MNDWI)
The MNDWI is used for detecting water bodies, making it particularly useful for water resource monitoring and flood detection.

$$
\text{MNDWI} = \frac{(B3 - B11)}{(B3 + B11)}
$$

Where:
- \(B3\) = Green band (Band 3)
- \(B11\) = Shortwave Infrared (SWIR1) band (Band 11)

In [9]:
# Calculate EVI
train_concat['EVI'] = 2.5 * (train_concat['B08'] - train_concat['B04']) / (train_concat['B08'] + 6 * train_concat['B04'] - 7.5 * train_concat['B02'] + 1)
validation_concat['EVI'] = 2.5 * (validation_concat['B08'] - validation_concat['B04']) / (validation_concat['B08'] + 6 * validation_concat['B04'] - 7.5 * validation_concat['B02'] + 1)

# Calculate SAVI
train_concat['SAVI'] = (train_concat['B08'] - train_concat['B04']) / (train_concat['B08'] + train_concat['B04'] + 0.5) * (1 + 0.5)
validation_concat['SAVI'] = (validation_concat['B08'] - validation_concat['B04']) / (validation_concat['B08'] + validation_concat['B04'] + 0.5) * (1 + 0.5)

# Calculate MNDWI
train_concat['MNDWI'] = (train_concat['B03'] - train_concat['B11']) / (train_concat['B03'] + train_concat['B11'])
validation_concat['MNDWI'] = (validation_concat['B03'] - validation_concat['B11']) / (validation_concat['B03'] + validation_concat['B11'])


In [10]:
train_concat.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,...,B8A,B11,B12,NDVI,NDBI,NDWI,LST,EVI,SAVI,MNDWI
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,825.0,990.0,1144.0,1172.0,1435.0,...,1688.0,1788.0,1540.0,0.219187,-0.011609,-0.230666,38.393941,1.143950,0.328726,-0.219645
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,825.0,990.0,1144.0,1172.0,1435.0,...,1688.0,1788.0,1540.0,0.219187,-0.011609,-0.230666,38.393941,1.143950,0.328726,-0.219645
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,825.0,622.0,785.0,744.0,1114.0,...,2353.0,1566.0,1170.0,0.514043,-0.193615,-0.494038,37.785534,1.857885,0.770939,-0.332199
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,825.0,619.0,750.0,738.0,1114.0,...,2353.0,1566.0,1170.0,0.514474,-0.190279,-0.508519,37.785534,1.872157,0.771584,-0.352332
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,825.0,556.0,758.0,660.0,1056.0,...,2259.0,1658.0,1240.0,0.551020,-0.157948,-0.500987,37.358281,1.955577,0.826390,-0.372517


In [11]:
validation_concat.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,...,B8A,B11,B12,NDVI,NDBI,NDWI,LST,EVI,SAVI,MNDWI
0,-73.971665,40.788763,NaN,12,794.0,511.0,568.0,527.0,1112.0,2323.0,...,2784.0,1741.0,1218.0,0.634915,-0.150939,-0.612022,36.202990,2.710736,0.952208,-0.508012
1,-73.971928,40.788875,NaN,12,1221.0,494.0,661.0,497.0,1315.0,2652.0,...,3102.0,2124.0,1541.0,0.761000,-0.265814,-0.694194,36.202990,2.691327,1.141363,-0.525314
2,-73.967080,40.789080,NaN,4,1049.0,841.0,968.0,1056.0,1028.0,1418.0,...,1636.0,1634.0,1198.0,0.140765,0.076416,-0.183122,36.069687,0.604261,0.211104,-0.255957
3,-73.972550,40.789082,NaN,12,1189.0,903.0,984.0,1108.0,1971.0,1978.0,...,2049.0,2303.0,2219.0,0.143078,0.218196,-0.200650,36.886594,0.682909,0.214576,-0.401278
4,-73.969697,40.787953,NaN,9,1018.0,716.0,1046.0,917.0,1286.0,2382.0,...,2912.0,2102.0,1705.0,0.536283,-0.182101,-0.487757,34.500816,1.672185,0.804323,-0.335451


## 4. Green Space with 200m buffer

In [12]:
train_green = pd.read_csv("/kaggle/input/green-space-without-recreational-ground/train_df_with_green_space (4).csv")
train_green.head()

,Longitude,Latitude,datetime,UHI Index,green_space
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,2
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,2
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,2
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,2
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,2


In [13]:
validation_green = pd.read_csv("/kaggle/input/green-space-without-recreational-ground/valid_df_with_green_space (4).csv")
validation_green.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,green_space
0,-73.971665,40.788763,NaN,2
1,-73.971928,40.788875,NaN,2
2,-73.967080,40.789080,NaN,1
3,-73.972550,40.789082,NaN,2
4,-73.969697,40.787953,NaN,1


In [14]:
train_green = train_green.drop(["Longitude","Latitude","datetime","UHI Index"], axis=1)
train_concat1 = pd.concat([train_concat, train_green], axis = 1)
train_concat1.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,...,B11,B12,NDVI,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,825.0,990.0,1144.0,1172.0,1435.0,...,1788.0,1540.0,0.219187,-0.011609,-0.230666,38.393941,1.143950,0.328726,-0.219645,2
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,825.0,990.0,1144.0,1172.0,1435.0,...,1788.0,1540.0,0.219187,-0.011609,-0.230666,38.393941,1.143950,0.328726,-0.219645,2
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,825.0,622.0,785.0,744.0,1114.0,...,1566.0,1170.0,0.514043,-0.193615,-0.494038,37.785534,1.857885,0.770939,-0.332199,2
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,825.0,619.0,750.0,738.0,1114.0,...,1566.0,1170.0,0.514474,-0.190279,-0.508519,37.785534,1.872157,0.771584,-0.352332,2
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,825.0,556.0,758.0,660.0,1056.0,...,1658.0,1240.0,0.551020,-0.157948,-0.500987,37.358281,1.955577,0.826390,-0.372517,2


In [15]:
train_concat1.shape

(11229, 24)

In [16]:
validation_green = validation_green.drop(["Longitude","Latitude","UHI Index"], axis=1)
validation_concat1 = pd.concat([validation_concat, validation_green], axis = 1)
validation_concat1.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,...,B11,B12,NDVI,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space
0,-73.971665,40.788763,NaN,12,794.0,511.0,568.0,527.0,1112.0,2323.0,...,1741.0,1218.0,0.634915,-0.150939,-0.612022,36.202990,2.710736,0.952208,-0.508012,2
1,-73.971928,40.788875,NaN,12,1221.0,494.0,661.0,497.0,1315.0,2652.0,...,2124.0,1541.0,0.761000,-0.265814,-0.694194,36.202990,2.691327,1.141363,-0.525314,2
2,-73.967080,40.789080,NaN,4,1049.0,841.0,968.0,1056.0,1028.0,1418.0,...,1634.0,1198.0,0.140765,0.076416,-0.183122,36.069687,0.604261,0.211104,-0.255957,1
3,-73.972550,40.789082,NaN,12,1189.0,903.0,984.0,1108.0,1971.0,1978.0,...,2303.0,2219.0,0.143078,0.218196,-0.200650,36.886594,0.682909,0.214576,-0.401278,2
4,-73.969697,40.787953,NaN,9,1018.0,716.0,1046.0,917.0,1286.0,2382.0,...,2102.0,1705.0,0.536283,-0.182101,-0.487757,34.500816,1.672185,0.804323,-0.335451,1


In [17]:
validation_concat1.shape

(1040, 23)

## 5. Water feature

In [18]:
train_water = pd.read_csv("/kaggle/input/water-feature-450m-buffer/train_df_with_water.csv")
train_water.head()

,Longitude,Latitude,datetime,UHI Index,water
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,0
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,0
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,0
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,0
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,0


In [19]:
validation_water = pd.read_csv("/kaggle/input/water-feature-450m-buffer/valid_df_with_water.csv")
validation_water.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,water
0,-73.971665,40.788763,NaN,0
1,-73.971928,40.788875,NaN,0
2,-73.967080,40.789080,NaN,1
3,-73.972550,40.789082,NaN,0
4,-73.969697,40.787953,NaN,1


In [20]:
train_water = train_water.drop(["Longitude","Latitude","datetime","UHI Index"], axis=1)
train_concat2 = pd.concat([train_concat1, train_water], axis = 1)
train_concat2.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,...,B12,NDVI,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space,water
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,825.0,990.0,1144.0,1172.0,1435.0,...,1540.0,0.219187,-0.011609,-0.230666,38.393941,1.143950,0.328726,-0.219645,2,0
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,825.0,990.0,1144.0,1172.0,1435.0,...,1540.0,0.219187,-0.011609,-0.230666,38.393941,1.143950,0.328726,-0.219645,2,0
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,825.0,622.0,785.0,744.0,1114.0,...,1170.0,0.514043,-0.193615,-0.494038,37.785534,1.857885,0.770939,-0.332199,2,0
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,825.0,619.0,750.0,738.0,1114.0,...,1170.0,0.514474,-0.190279,-0.508519,37.785534,1.872157,0.771584,-0.352332,2,0
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,825.0,556.0,758.0,660.0,1056.0,...,1240.0,0.551020,-0.157948,-0.500987,37.358281,1.955577,0.826390,-0.372517,2,0


In [21]:
train_concat2.shape

(11229, 25)

In [22]:
validation_water = validation_water.drop(["Longitude","Latitude","UHI Index"], axis=1)
validation_concat2 = pd.concat([validation_concat1, validation_water], axis = 1)
validation_concat2.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,...,B12,NDVI,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space,water
0,-73.971665,40.788763,NaN,12,794.0,511.0,568.0,527.0,1112.0,2323.0,...,1218.0,0.634915,-0.150939,-0.612022,36.202990,2.710736,0.952208,-0.508012,2,0
1,-73.971928,40.788875,NaN,12,1221.0,494.0,661.0,497.0,1315.0,2652.0,...,1541.0,0.761000,-0.265814,-0.694194,36.202990,2.691327,1.141363,-0.525314,2,0
2,-73.967080,40.789080,NaN,4,1049.0,841.0,968.0,1056.0,1028.0,1418.0,...,1198.0,0.140765,0.076416,-0.183122,36.069687,0.604261,0.211104,-0.255957,1,1
3,-73.972550,40.789082,NaN,12,1189.0,903.0,984.0,1108.0,1971.0,1978.0,...,2219.0,0.143078,0.218196,-0.200650,36.886594,0.682909,0.214576,-0.401278,2,0
4,-73.969697,40.787953,NaN,9,1018.0,716.0,1046.0,917.0,1286.0,2382.0,...,1705.0,0.536283,-0.182101,-0.487757,34.500816,1.672185,0.804323,-0.335451,1,1


In [23]:
validation_concat2.shape

(1040, 24)

## 6. Road Density

In [24]:
train_road = pd.read_csv("/kaggle/input/road-density-300m-without-primary/train_df_with_road_density (4).csv")
train_road.head()

,Longitude,Latitude,datetime,UHI Index,road_density
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,636.198715
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,636.198715
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,636.198715
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,636.198715
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,636.198715


In [25]:
validation_road = pd.read_csv("/kaggle/input/road-density-300m-without-primary/valid_df_with_road_density (4).csv")
validation_road.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,road_density
0,-73.971665,40.788763,NaN,0.000000
1,-73.971928,40.788875,NaN,0.000000
2,-73.967080,40.789080,NaN,682.714949
3,-73.972550,40.789082,NaN,0.000000
4,-73.969697,40.787953,NaN,526.855898


In [26]:
train_road = train_road.drop(["Longitude","Latitude","datetime","UHI Index"], axis=1)
train_concat3 = pd.concat([train_concat2, train_road], axis = 1)
train_concat3.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,...,NDVI,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space,water,road_density
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,825.0,990.0,1144.0,1172.0,1435.0,...,0.219187,-0.011609,-0.230666,38.393941,1.143950,0.328726,-0.219645,2,0,636.198715
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,825.0,990.0,1144.0,1172.0,1435.0,...,0.219187,-0.011609,-0.230666,38.393941,1.143950,0.328726,-0.219645,2,0,636.198715
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,825.0,622.0,785.0,744.0,1114.0,...,0.514043,-0.193615,-0.494038,37.785534,1.857885,0.770939,-0.332199,2,0,636.198715
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,825.0,619.0,750.0,738.0,1114.0,...,0.514474,-0.190279,-0.508519,37.785534,1.872157,0.771584,-0.352332,2,0,636.198715
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,825.0,556.0,758.0,660.0,1056.0,...,0.551020,-0.157948,-0.500987,37.358281,1.955577,0.826390,-0.372517,2,0,636.198715


In [27]:
train_concat3.shape

(11229, 26)

In [28]:
validation_road = validation_road.drop(["Longitude","Latitude","UHI Index"], axis=1)
validation_concat3 = pd.concat([validation_concat2, validation_road], axis = 1)
validation_concat3.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,...,NDVI,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space,water,road_density
0,-73.971665,40.788763,NaN,12,794.0,511.0,568.0,527.0,1112.0,2323.0,...,0.634915,-0.150939,-0.612022,36.202990,2.710736,0.952208,-0.508012,2,0,0.000000
1,-73.971928,40.788875,NaN,12,1221.0,494.0,661.0,497.0,1315.0,2652.0,...,0.761000,-0.265814,-0.694194,36.202990,2.691327,1.141363,-0.525314,2,0,0.000000
2,-73.967080,40.789080,NaN,4,1049.0,841.0,968.0,1056.0,1028.0,1418.0,...,0.140765,0.076416,-0.183122,36.069687,0.604261,0.211104,-0.255957,1,1,682.714949
3,-73.972550,40.789082,NaN,12,1189.0,903.0,984.0,1108.0,1971.0,1978.0,...,0.143078,0.218196,-0.200650,36.886594,0.682909,0.214576,-0.401278,2,0,0.000000
4,-73.969697,40.787953,NaN,9,1018.0,716.0,1046.0,917.0,1286.0,2382.0,...,0.536283,-0.182101,-0.487757,34.500816,1.672185,0.804323,-0.335451,1,1,526.855898


In [29]:
validation_concat3.shape

(1040, 25)

## 7. Building density 2

In [30]:
train_density2 = pd.read_csv("/kaggle/input/building-density2-taller-than-18m-500m/train_df_with_building_density2 (3).csv")
train_density2.head()

,Longitude,Latitude,datetime,UHI Index,building_density2
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,51
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,51
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,50
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,48
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,46


In [31]:
validation_density2 = pd.read_csv("/kaggle/input/building-density2-taller-than-18m-500m/valid_df_with_building_density2 (3).csv")
validation_density2.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,building_density2
0,-73.971665,40.788763,NaN,348
1,-73.971928,40.788875,NaN,351
2,-73.967080,40.789080,NaN,187
3,-73.972550,40.789082,NaN,333
4,-73.969697,40.787953,NaN,312


In [32]:
train_density2 = train_density2.drop(["Longitude","Latitude","datetime","UHI Index"], axis=1)
train_concat4 = pd.concat([train_concat3, train_density2], axis = 1)
train_concat4.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,...,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space,water,road_density,building_density2
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,825.0,990.0,1144.0,1172.0,1435.0,...,-0.011609,-0.230666,38.393941,1.143950,0.328726,-0.219645,2,0,636.198715,51
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,825.0,990.0,1144.0,1172.0,1435.0,...,-0.011609,-0.230666,38.393941,1.143950,0.328726,-0.219645,2,0,636.198715,51
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,825.0,622.0,785.0,744.0,1114.0,...,-0.193615,-0.494038,37.785534,1.857885,0.770939,-0.332199,2,0,636.198715,50
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,825.0,619.0,750.0,738.0,1114.0,...,-0.190279,-0.508519,37.785534,1.872157,0.771584,-0.352332,2,0,636.198715,48
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,825.0,556.0,758.0,660.0,1056.0,...,-0.157948,-0.500987,37.358281,1.955577,0.826390,-0.372517,2,0,636.198715,46


In [33]:
train_concat4.shape

(11229, 27)

In [34]:
validation_density2 = validation_density2.drop(["Longitude","Latitude","UHI Index"], axis=1)
validation_concat4 = pd.concat([validation_concat3, validation_density2], axis = 1)
validation_concat4.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,...,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space,water,road_density,building_density2
0,-73.971665,40.788763,NaN,12,794.0,511.0,568.0,527.0,1112.0,2323.0,...,-0.150939,-0.612022,36.202990,2.710736,0.952208,-0.508012,2,0,0.000000,348
1,-73.971928,40.788875,NaN,12,1221.0,494.0,661.0,497.0,1315.0,2652.0,...,-0.265814,-0.694194,36.202990,2.691327,1.141363,-0.525314,2,0,0.000000,351
2,-73.967080,40.789080,NaN,4,1049.0,841.0,968.0,1056.0,1028.0,1418.0,...,0.076416,-0.183122,36.069687,0.604261,0.211104,-0.255957,1,1,682.714949,187
3,-73.972550,40.789082,NaN,12,1189.0,903.0,984.0,1108.0,1971.0,1978.0,...,0.218196,-0.200650,36.886594,0.682909,0.214576,-0.401278,2,0,0.000000,333
4,-73.969697,40.787953,NaN,9,1018.0,716.0,1046.0,917.0,1286.0,2382.0,...,-0.182101,-0.487757,34.500816,1.672185,0.804323,-0.335451,1,1,526.855898,312


In [35]:
validation_concat4.shape

(1040, 26)

# Select features 

In [36]:
features = ['B01','LST','density','green_space','water','road_density','building_density2']
train_df = train_concat4[features + ["UHI Index"]]
train_df

,B01,LST,density,green_space,water,road_density,building_density2,UHI Index
0,825.0,38.393941,8,2,0,636.198715,51,1.030289
1,825.0,38.393941,8,2,0,636.198715,51,1.030289
2,825.0,37.785534,8,2,0,636.198715,50,1.023798
3,825.0,37.785534,8,2,0,636.198715,48,1.023798
4,825.0,37.358281,8,2,0,636.198715,46,1.021634
...,...,...,...,...,...,...,...,...
11224,432.0,30.293234,0,1,1,1953.022363,45,0.972470
11225,432.0,30.293234,0,1,1,1953.022363,46,0.972470
11226,432.0,30.440209,0,1,1,1953.022363,46,0.981124
11227,432.0,30.440209,0,1,1,1953.022363,46,0.981245


# Remove duplicates from training data

In [37]:
# Remove duplicate rows from the DataFrame based on specified columns and keep the first occurrence
for col in features:
    # Check if the value is a numpy array and has more than one dimension
    train_df[col] = train_df[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)

# Now remove duplicates
uhi_data = train_df.drop_duplicates(subset=features, keep='first')
uhi_data.head()

<ipython-input-37-cc52a2d8b8e0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[col] = train_df[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)


,B01,LST,density,green_space,water,road_density,building_density2,UHI Index
0,825.0,38.393941,8,2,0,636.198715,51,1.030289
2,825.0,37.785534,8,2,0,636.198715,50,1.023798
3,825.0,37.785534,8,2,0,636.198715,48,1.023798
4,825.0,37.358281,8,2,0,636.198715,46,1.021634
5,825.0,37.358281,8,1,0,636.198715,46,1.021634


In [38]:
uhi_data.shape

(9411, 8)

In [39]:
# Resetting the index of the dataset
uhi_data=uhi_data.reset_index(drop=True)

In [40]:
uhi_data.isna().sum()

B01                  0
LST                  0
density              0
green_space          0
water                0
road_density         0
building_density2    0
UHI Index            0
dtype: int64

# Model Building

In [41]:
# Split the data into features (X) and target (y), and then into training and testing sets
X = uhi_data.drop(columns=['UHI Index']).values
y = uhi_data ['UHI Index'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

**Feature Scaling**

In [42]:
# Scale the training and test data using standardscaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Model Training**

TabPFN

In [43]:
!pip install tabpfn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 4.8 MB/s eta 0:00:00


In [44]:
from tabpfn import TabPFNRegressor

# Initialize the TabPFN model and name it best_pfn
best_pfn = TabPFNRegressor(device="cuda")  # Use "cuda" for GPU or "cpu" for CPU

# Fit the model
best_pfn.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/tabpfn/base.py:100: UserWarning: Downloading model to /root/.cache/tabpfn/tabpfn-v2-regressor.ckpt.
  model, bardist, config_ = load_model_criterion_config(


tabpfn-v2-regressor.ckpt:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

TabPFNRegressor(device='cuda')

**Model Evaluation**

In [45]:
# Evaluate the ensemble on the test set
final_r2 = r2_score(y_test, best_pfn.predict(X_test))
print(f"Final R² score: {final_r2:.4f}")

Final R² score: 0.9434


# Submission

In [46]:
validation_concat4.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,...,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space,water,road_density,building_density2
0,-73.971665,40.788763,NaN,12,794.0,511.0,568.0,527.0,1112.0,2323.0,...,-0.150939,-0.612022,36.202990,2.710736,0.952208,-0.508012,2,0,0.000000,348
1,-73.971928,40.788875,NaN,12,1221.0,494.0,661.0,497.0,1315.0,2652.0,...,-0.265814,-0.694194,36.202990,2.691327,1.141363,-0.525314,2,0,0.000000,351
2,-73.967080,40.789080,NaN,4,1049.0,841.0,968.0,1056.0,1028.0,1418.0,...,0.076416,-0.183122,36.069687,0.604261,0.211104,-0.255957,1,1,682.714949,187
3,-73.972550,40.789082,NaN,12,1189.0,903.0,984.0,1108.0,1971.0,1978.0,...,0.218196,-0.200650,36.886594,0.682909,0.214576,-0.401278,2,0,0.000000,333
4,-73.969697,40.787953,NaN,9,1018.0,716.0,1046.0,917.0,1286.0,2382.0,...,-0.182101,-0.487757,34.500816,1.672185,0.804323,-0.335451,1,1,526.855898,312


In [47]:
submission_val_data = validation_concat4[features]
submission_val_data.head()

,B01,LST,density,green_space,water,road_density,building_density2
0,794.0,36.202990,12,2,0,0.000000,348
1,1221.0,36.202990,12,2,0,0.000000,351
2,1049.0,36.069687,4,1,1,682.714949,187
3,1189.0,36.886594,12,2,0,0.000000,333
4,1018.0,34.500816,9,1,1,526.855898,312


In [48]:
# Feature Scaling 
submission_val_data = submission_val_data.values
transformed_submission_data = sc.transform(submission_val_data)

In [49]:
#Making predictions
final_predictions = best_pfn.predict(transformed_submission_data)
final_prediction_series = pd.Series(final_predictions)

In [50]:
#Combining the results into dataframe
submission_df = pd.DataFrame({'Longitude':validation_concat['Longitude'].values, 'Latitude':validation_concat['Latitude'].values, 'UHI Index':final_prediction_series.values})

In [51]:
#Displaying the sample submission dataframe
submission_df.head()

,Longitude,Latitude,UHI Index
0,-73.971665,40.788763,0.963253
1,-73.971928,40.788875,0.962759
2,-73.967080,40.789080,0.963206
3,-73.972550,40.789082,0.963176
4,-73.969697,40.787953,0.958092


In [52]:
#Dumping the predictions into a csv file.
submission_df.to_csv("submission.csv",index = False)